In [18]:
import requests
import re
import pandas as pd
from urllib.parse import quote
from datetime import datetime
import matplotlib.pyplot as plt

In [19]:
#Importartion of major events date during covid accros countries
data = pd.read_csv('data/interventions.csv')
data

,lang,1st case,1st death,School closure,Public events banned,Lockdown,Mobility,Normalcy
0,fr,2020-01-24,2020-02-14,2020-03-14,2020-03-13,2020-03-17,2020-03-16,2020-07-02
1,da,2020-02-27,2020-03-12,2020-03-13,2020-03-12,2020-03-18,2020-03-11,2020-06-05
2,de,2020-01-27,2020-03-09,2020-03-14,2020-03-22,2020-03-22,2020-03-16,2020-07-10
3,it,2020-01-31,2020-02-22,2020-03-05,2020-03-09,2020-03-11,2020-03-11,2020-06-26
4,nl,2020-02-27,2020-03-06,2020-03-11,2020-03-24,NaN,2020-03-16,2020-05-29
5,no,2020-02-26,2020-02-26,2020-03-13,2020-03-12,2020-03-24,2020-03-11,2020-06-04
6,sr,2020-03-06,2020-03-20,2020-03-15,2020-03-21,2020-03-21,2020-03-16,2020-05-02
7,sv,2020-01-31,2020-03-11,2020-03-18,2020-03-12,NaN,2020-03-11,2020-06-05
8,ko,2020-01-20,2020-02-20,2020-02-23,NaN,NaN,2020-02-25,2020-04-15
9,ca,2020-01-31,2020-02-13,2020-03-12,2020-03-08,2020-03-14,2020-03-16,NaN


In [20]:
# Define a user agent string
user_agent = 'MandarineCorp (clementine.naim@epfl.ch)'

# Specify the headers with the user agent
headers = {
    'User-Agent': user_agent,
    'accept': 'application/json'
}


In [34]:
# Specify the page name
page_name = ['Absurdism','Stoicism']

# Specify the languages you want to retrieve language links for
target_languages = ['fr', 'es']  # Add or remove language codes as needed

In [27]:
def get_url_list(target_languages, names):
    api_url = "https://en.wikipedia.org/w/api.php"
    language_links=[];

    for page in names : 

    # Make separate requests for each language
        for lang in target_languages:
            params = {
                'action': 'query',
                'titles': page,
                'prop': 'langlinks',
                'llprop': 'url',
                'format': 'json',
                'lllang': lang,
            }

            # Make the API request
            response = requests.get(api_url, params=params)
            data = response.json()

            # Check if the request was successful (status code 200)
            if response.status_code == 200:
                # Extract language links from the API response
                pages = data['query']['pages']
                page_id = next(iter(pages))
                langlinks = pages[page_id].get('langlinks', [])
                for link in langlinks:
                    language_links.append( link['url'])
            else:
                print(f"Error for language {lang}: {response.status_code}")
                
    return language_links
        
        

In [28]:
#finds the country code from a given link, used in  get_page_views_by_languages(links)
def find_country_code_and_title(link):
    state=0
    code=""
    title=""
    for i in range(len(link)):
        if state==2 and link[i]==".":
            state+=1
        if state==2:
            code+=link[i]
        if state==5:
            title+=link[i]
        if link[i]=="/":
            state+=1
    return code,title

In [59]:
def get_page_views_by_languages(links,names):
    philo_views=pd.DataFrame()
    i=0
    index_subject = 0
    for link in links:
        # Define a user agent to have acces to the API 
        user_agent = 'MandarineCorp (clementine.naim@epfl.ch)'
        # Specify the headers with the user agent
        headers = {
            'User-Agent': user_agent,
            'accept': 'application/json'
        
        }
        # Find country code:
        code, title = find_country_code_and_title(link)
        print(code,title)
        
        subject = names[index_subject]
        print(subject)
        url = f'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/{code}.wikipedia.org/all-access/all-agents/{title}/monthly/2018010100/2023010100'
        # Making a GET request
        response_views = requests.get(url,headers=headers)

        # Check if the request was successful (status code 200)
        if response_views.status_code == 200:
            # Print the response content
            data = response_views.json()
        else:
            # Print an error message if the request was unsuccessful
            print(f"Error: {response_views.status_code}")
            print(title)


        # Extract the 'items' list from the data
        items_list = data['items']
        # Create a DataFrame
        df = pd.DataFrame(items_list)
        df["subject"]=subject
        df["code"]=code
        philo_views = pd.concat([philo_views,df])

        i+=1
        if i==2 :
            index_subject+=1
            i=0
    return philo_views

# Finding how many visits there is per subject in the different languages relatively to each other

In [60]:
langs = get_url_list(target_languages,page_name)
langs

['https://fr.wikipedia.org/wiki/Absurde',
 'https://es.wikipedia.org/wiki/Filosof%C3%ADa_del_absurdo',
 'https://fr.wikipedia.org/wiki/Sto%C3%AFcisme',
 'https://es.wikipedia.org/wiki/Estoicismo']

In [65]:
df=get_page_views_by_languages(langs,page_name)
df.head(100)

fr Absurde
Absurdism
es Filosof%C3%ADa_del_absurdo
Absurdism
fr Sto%C3%AFcisme
Stoicism
es Estoicismo
Stoicism


,project,article,granularity,timestamp,access,agent,views,subject,code
0,fr.wikipedia,Absurde,monthly,2018010100,all-access,all-agents,10119,Absurdism,fr
1,fr.wikipedia,Absurde,monthly,2018020100,all-access,all-agents,8507,Absurdism,fr
2,fr.wikipedia,Absurde,monthly,2018030100,all-access,all-agents,9485,Absurdism,fr
3,fr.wikipedia,Absurde,monthly,2018040100,all-access,all-agents,9772,Absurdism,fr
4,fr.wikipedia,Absurde,monthly,2018050100,all-access,all-agents,11073,Absurdism,fr
...,...,...,...,...,...,...,...,...,...
34,es.wikipedia,Filosofía_del_absurdo,monthly,2020110100,all-access,all-agents,9769,Absurdism,es
35,es.wikipedia,Filosofía_del_absurdo,monthly,2020120100,all-access,all-agents,7816,Absurdism,es
36,es.wikipedia,Filosofía_del_absurdo,monthly,2021010100,all-access,all-agents,7422,Absurdism,es
37,es.wikipedia,Filosofía_del_absurdo,monthly,2021020100,all-access,all-agents,7383,Absurdism,es


In [66]:
#finding total visits over 5 years:
dfsum=df.groupby(['article','code','subject']).sum('views')
dfsum.head(500)


,,,views
article,code,subject,
Absurde,fr,Absurdism,423476
Estoicismo,es,Stoicism,3404270
Filosofía_del_absurdo,es,Absurdism,470835
Stoïcisme,fr,Stoicism,1794164


In [12]:
def get_url_list(target_languages, page_,name):
    api_url = "https://en.wikipedia.org/w/api.php"

    for page in page_titles : 

    # Make separate requests for each language
        for lang in target_languages:
            params = {
                'action': 'query',
                'titles': page,
                'prop': 'langlinks',
                'llprop': 'url',
                'format': 'json',
                'lllang': lang,
            }

            # Make the API request
            response = requests.get(api_url, params=params)
            data = response.json()

            # Check if the request was successful (status code 200)
            if response.status_code == 200:
                # Extract language links from the API response
                pages = data['query']['pages']
                page_id = next(iter(pages))
                langlinks = pages[page_id].get('langlinks', [])
                for link in langlinks:
                    language_links.append( link['url'])
            else:
                print(f"Error for language {lang}: {response.status_code}")
    return language_links
        
        